# COCO Image Viewer
This notebook will allow you to view details about a COCO dataset and preview segmentations on annotated images.
Learn more about it at: http://cocodataset.org/

In [1]:
import IPython
import os
import json
import random
import numpy as np
import requests
from io import BytesIO
from math import trunc
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

In [2]:
# Load the dataset json
class CocoDataset():
    def __init__(self, annotation_path, image_dir):
        self.annotation_path = annotation_path
        self.image_dir = image_dir
        self.colors = ['blue', 'purple', 'red', 'green', 'orange', 'salmon', 'pink', 'gold',
                       'orchid', 'slateblue', 'limegreen', 'seagreen', 'darkgreen', 'olive',
                       'teal', 'aquamarine', 'steelblue', 'powderblue', 'dodgerblue', 'navy',
                       'magenta', 'sienna', 'maroon']

        json_file = open(self.annotation_path)
        self.coco = json.load(json_file)
        json_file.close()

        self.process_info()
        self.process_licenses()
        self.process_categories()
        self.process_images()
        self.process_segmentations()

    def display_info(self):
        print('Dataset Info:')
        print('=============')
        if self.info is None:
            return
        for key, item in self.info.items():
            print('  {}: {}'.format(key, item))

        requirements = [['description', str],
                        ['url', str],
                        ['version', str],
                        ['year', int],
                        ['contributor', str],
                        ['date_created', str]]
        for req, req_type in requirements:
            if req not in self.info:
                print('ERROR: {} is missing'.format(req))
            elif type(self.info[req]) != req_type:
                print('ERROR: {} should be type {}'.format(req, str(req_type)))
        print('')

    def display_licenses(self):
        print('Licenses:')
        print('=========')

        if self.licenses is None:
            return
        requirements = [['id', int],
                        ['url', str],
                        ['name', str]]
        for license in self.licenses:
            for key, item in license.items():
                print('  {}: {}'.format(key, item))
            for req, req_type in requirements:
                if req not in license:
                    print('ERROR: {} is missing'.format(req))
                elif type(license[req]) != req_type:
                    print('ERROR: {} should be type {}'.format(
                        req, str(req_type)))
            print('')
        print('')

    def display_categories(self):
        print('Categories:')
        print('=========')
        for sc_key, sc_val in self.super_categories.items():
            print('  super_category: {}'.format(sc_key))
            for cat_id in sc_val:
                print('    id {}: {}'.format(
                    cat_id, self.categories[cat_id]['name']))
            print('')

    def display_image(self, image_id, show_polys=True, show_bbox=True, show_crowds=True, use_url=False):
        print('Image:')
        print('======')
        if image_id == 'random':
            image_id = random.choice(list(self.images.keys()))

        # Print the image info
        image = self.images[image_id]
        for key, val in image.items():
            print('  {}: {}'.format(key, val))

        # Open the image
        if use_url:
            image_path = image['coco_url']
            response = requests.get(image_path)
            image = PILImage.open(BytesIO(response.content))

        else:
            # image_path = os.path.join(self.image_dir, image['file_name'])
            image_path = "{}/{}".format(self.image_dir, image['file_name'])
            image = PILImage.open(image_path)

        # Calculate the size and adjusted display size
        max_width = 600
        image_width, image_height = image.size
        adjusted_width = min(image_width, max_width)
        adjusted_ratio = adjusted_width / image_width
        adjusted_height = adjusted_ratio * image_height

        # Create list of polygons to be drawn
        polygons = {}
        bbox_polygons = {}
        rle_regions = {}
        poly_colors = {}
        bbox_categories = {}
        print('  segmentations ({}):'.format(
            len(self.segmentations[image_id])))
        for i, segm in enumerate(self.segmentations[image_id]):
            polygons_list = []
            if segm['iscrowd'] != 0:
                # Gotta decode the RLE
                px = 0
                x, y = 0, 0
                rle_list = []
                for j, counts in enumerate(segm['segmentation']['counts']):
                    if j % 2 == 0:
                        # Empty pixels
                        px += counts
                    else:
                        # Need to draw on these pixels, since we are drawing in vector form,
                        # we need to draw horizontal lines on the image
                        x_start = trunc(
                            trunc(px / image_height) * adjusted_ratio)
                        y_start = trunc(px % image_height * adjusted_ratio)
                        px += counts
                        x_end = trunc(trunc(px / image_height)
                                      * adjusted_ratio)
                        y_end = trunc(px % image_height * adjusted_ratio)
                        if x_end == x_start:
                            # This is only on one line
                            rle_list.append(
                                {'x': x_start, 'y': y_start, 'width': 1, 'height': (y_end - y_start)})
                        if x_end > x_start:
                            # This spans more than one line
                            # Insert top line first
                            rle_list.append(
                                {'x': x_start, 'y': y_start, 'width': 1, 'height': (image_height - y_start)})

                            # Insert middle lines if needed
                            lines_spanned = x_end - x_start + 1  # total number of lines spanned
                            full_lines_to_insert = lines_spanned - 2
                            if full_lines_to_insert > 0:
                                full_lines_to_insert = trunc(
                                    full_lines_to_insert * adjusted_ratio)
                                rle_list.append(
                                    {'x': (x_start + 1), 'y': 0, 'width': full_lines_to_insert, 'height': image_height})

                            # Insert bottom line
                            rle_list.append(
                                {'x': x_end, 'y': 0, 'width': 1, 'height': y_end})
                if len(rle_list) > 0:
                    rle_regions[segm['id']] = rle_list
            else:
                # Add the polygon segmentation
                for segmentation_points in segm['segmentation']:
                    segmentation_points = np.multiply(
                        segmentation_points, adjusted_ratio).astype(int)
                    polygons_list.append(
                        str(segmentation_points).lstrip('[').rstrip(']'))
            polygons[segm['id']] = polygons_list
            if i < len(self.colors):
                poly_colors[segm['id']] = self.colors[i]
            else:
                poly_colors[segm['id']] = 'white'

            bbox = segm['bbox']
            bbox_points = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1],
                           bbox[0] + bbox[2], bbox[1] +
                           bbox[3], bbox[0], bbox[1] + bbox[3],
                           bbox[0], bbox[1]]
            bbox_points = np.multiply(bbox_points, adjusted_ratio).astype(int)
            bbox_polygons[segm['id']] = str(
                bbox_points).lstrip('[').rstrip(']')
            bbox_categories[segm['id']] = self.categories[segm['category_id']]
            # Print details
            print('    {}:{}:{}'.format(
                segm['id'], poly_colors[segm['id']], self.categories[segm['category_id']]))

        # Draw segmentation polygons on image
        html = '<div class="container" style="position:relative;">'
        html += '<img src="{}" style="position:relative;top:0px;left:0px;width:{}px;">'.format(
            image_path, adjusted_width)
        html += '<div class="svgclass"><svg width="{}" height="{}">'.format(
            adjusted_width, adjusted_height)

        if show_polys:
            for seg_id, points_list in polygons.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for points in points_list:
                    html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5" />'.format(
                        points, fill_color, stroke_color)

        if show_crowds:
            for seg_id, rect_list in rle_regions.items():
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                for rect_def in rect_list:
                    x, y = rect_def['x'], rect_def['y']
                    w, h = rect_def['width'], rect_def['height']
                    html += '<rect x="{}" y="{}" width="{}" height="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0.5; stroke-opacity:0.5" />'.format(
                        x, y, w, h, fill_color, stroke_color)

        if show_bbox:
            for seg_id, points in bbox_polygons.items():
                x, y = [int(i) for i in points.split()[:2]]
                html += '<text x="{}" y="{}" fill="yellow">{}</text>'.format(
                    x, y, bbox_categories[seg_id]["name"])
                fill_color = poly_colors[seg_id]
                stroke_color = poly_colors[seg_id]
                html += '<polygon points="{}" style="fill:{}; stroke:{}; stroke-width:1; fill-opacity:0" />'.format(
                    points, fill_color, stroke_color)

        html += '</svg></div>'
        html += '</div>'
        html += '<style>'
        html += '.svgclass { position:absolute; top:0px; left:0px;}'
        html += '</style>'
        return html

    def process_info(self):
        self.info = self.coco.get('info')

    def process_licenses(self):
        self.licenses = self.coco.get('licenses')

    def process_categories(self):
        self.categories = {}
        self.super_categories = {}
        for category in self.coco['categories']:
            cat_id = category['id']
            super_category = category['supercategory']

            # Add category to the categories dict
            if cat_id not in self.categories:
                self.categories[cat_id] = category
            else:
                print("ERROR: Skipping duplicate category id: {}".format(category))

            # Add category to super_categories dict
            if super_category not in self.super_categories:
                # Create a new set with the category id
                self.super_categories[super_category] = {cat_id}
            else:
                self.super_categories[super_category] |= {
                    cat_id}  # Add category id to the set

    def process_images(self):
        self.images = {}
        for image in self.coco['images']:
            image_id = image['id']
            if image_id in self.images:
                print("ERROR: Skipping duplicate image id: {}".format(image))
            else:
                self.images[image_id] = image

    def process_segmentations(self):
        self.segmentations = {}
        for segmentation in self.coco['annotations']:
            image_id = segmentation['image_id']
            if image_id not in self.segmentations:
                self.segmentations[image_id] = []
            self.segmentations[image_id].append(segmentation)

In [3]:
annotation_path = "trainval.json"
image_dir = "images"

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_info()
coco_dataset.display_licenses()
coco_dataset.display_categories()

Dataset Info:
Licenses:
Categories:
  super_category: Animal
    id 0: Animal

  super_category: Building
    id 1: Building

  super_category: Bus
    id 2: Bus

  super_category: Car
    id 3: Car

  super_category: Cartoon
    id 4: Cartoon

  super_category: Design
    id 5: Design

  super_category: Graffiti
    id 6: Graffiti

  super_category: Human
    id 7: Human

  super_category: Nature
    id 8: Nature

  super_category: No Car
    id 9: No Car

  super_category: Tatto
    id 10: Tatto

  super_category: Tex
    id 11: Tex

  super_category: Text
    id 12: Text

  super_category: Truck
    id 13: Truck

  super_category: Ttuck
    id 14: Ttuck

  super_category: Underpass
    id 15: Underpass

  super_category: Wall
    id 16: Wall



In [4]:
for k, v in coco_dataset.images.items():
    print(k, v)

0 {'height': 300, 'width': 300, 'id': 0, 'file_name': '2014-10-23 23.42.37 838130451809134694_cargraffiti.jpg'}
1 {'height': 300, 'width': 300, 'id': 1, 'file_name': '2015-12-01 12.26.07 1130629554560784642_cargraffiti.jpg'}
2 {'height': 300, 'width': 300, 'id': 2, 'file_name': '2017-07-24 23.04.04 1566510663067939027_cargraffiti.jpg'}
3 {'height': 300, 'width': 300, 'id': 3, 'file_name': '2018-06-14 00.03.41 1801368002497480325_cargraffiti.jpg'}
4 {'height': 300, 'width': 300, 'id': 4, 'file_name': '2018-06-30 21.13.41 1813603622047486250_cargraffiti.jpg'}
5 {'height': 300, 'width': 300, 'id': 5, 'file_name': '2018-07-04 03.11.31 1815958053695853818_cargraffiti.jpg'}
6 {'height': 300, 'width': 300, 'id': 6, 'file_name': '2018-07-04 21.31.15 1816511570894998900_cargraffiti.jpg'}
7 {'height': 300, 'width': 300, 'id': 7, 'file_name': '2018-07-09 02.29.57 1819561018075764768_cargraffiti.jpg'}
8 {'height': 300, 'width': 300, 'id': 8, 'file_name': '2018-07-10 09.09.33 1820486912118852148_ca

409 {'height': 928, 'width': 1080, 'id': 409, 'file_name': '2019-04-12_09-23-08_UTC.jpg'}
410 {'height': 1080, 'width': 1080, 'id': 410, 'file_name': '2019-04-12_09-33-50_UTC.jpg'}
411 {'height': 1349, 'width': 1080, 'id': 411, 'file_name': '2019-04-15_11-41-21_UTC.jpg'}
412 {'height': 910, 'width': 1080, 'id': 412, 'file_name': '2019-04-19_18-47-25_UTC.jpg'}
413 {'height': 953, 'width': 1080, 'id': 413, 'file_name': '2019-05-05_02-24-14_UTC.jpg'}
414 {'height': 1080, 'width': 1080, 'id': 414, 'file_name': '2019-05-07_10-06-33_UTC.jpg'}
415 {'height': 900, 'width': 720, 'id': 415, 'file_name': '2019-10-28_01-47-39_UTC.jpg'}
416 {'height': 1214, 'width': 1080, 'id': 416, 'file_name': '2019-05-10_01-09-02_UTC.jpg'}
417 {'height': 809, 'width': 1080, 'id': 417, 'file_name': '2019-05-14_01-13-47_UTC.jpg'}
418 {'height': 900, 'width': 720, 'id': 418, 'file_name': '2019-06-01_04-22-44_UTC.jpg'}
419 {'height': 1080, 'width': 1080, 'id': 419, 'file_name': '2019-06-25_16-44-35_UTC.jpg'}
420 {'h

573 {'height': 760, 'width': 760, 'id': 573, 'file_name': '2019-10-13_13-48-57_UTC_3.jpg'}
574 {'height': 760, 'width': 760, 'id': 574, 'file_name': '2019-10-13_13-48-57_UTC_4.jpg'}
575 {'height': 760, 'width': 760, 'id': 575, 'file_name': '2019-10-13_13-48-57_UTC_5.jpg'}
576 {'height': 760, 'width': 760, 'id': 576, 'file_name': '2019-10-13_13-48-57_UTC_6.jpg'}
577 {'height': 406, 'width': 720, 'id': 577, 'file_name': '2019-10-13_13-52-47_UTC.jpg'}
578 {'height': 721, 'width': 1079, 'id': 578, 'file_name': '2019-10-13_14-58-26_UTC_1.jpg'}
579 {'height': 722, 'width': 1080, 'id': 579, 'file_name': '2019-10-13_14-58-26_UTC_2.jpg'}
580 {'height': 722, 'width': 1080, 'id': 580, 'file_name': '2019-10-13_14-58-26_UTC_3.jpg'}
581 {'height': 720, 'width': 1080, 'id': 581, 'file_name': '2019-10-28_09-05-55_UTC_1.jpg'}
582 {'height': 1080, 'width': 1080, 'id': 582, 'file_name': '2019-10-13_15-09-38_UTC.jpg'}
583 {'height': 1080, 'width': 1080, 'id': 583, 'file_name': '2019-10-14_04-25-14_UTC.jpg

In [13]:
html = coco_dataset.display_image(379, use_url=False)
IPython.display.HTML(html)

Image:
  height: 300
  width: 300
  id: 379
  file_name: 2019-04-03 10.49.56 2014052559183167619_cargraffiti.jpg
  segmentations (7):
    1427:blue:{'supercategory': 'Car', 'id': 3, 'name': 'Car'}
    1428:purple:{'supercategory': 'Graffiti', 'id': 6, 'name': 'Graffiti'}
    1429:red:{'supercategory': 'Text', 'id': 12, 'name': 'Text'}
    1430:green:{'supercategory': 'Text', 'id': 12, 'name': 'Text'}
    1431:orange:{'supercategory': 'Text', 'id': 12, 'name': 'Text'}
    1432:salmon:{'supercategory': 'Text', 'id': 12, 'name': 'Text'}
    1433:pink:{'supercategory': 'Wall', 'id': 16, 'name': 'Wall'}


In [15]:
html = coco_dataset.display_image(286, use_url=False)
IPython.display.HTML(html)

Image:
  height: 225
  width: 300
  id: 286
  file_name: 2019-01-23 15.37.12 1963489875261213201_cargraffiti.jpg
  segmentations (4):
    1082:blue:{'supercategory': 'Car', 'id': 3, 'name': 'Car'}
    1083:purple:{'supercategory': 'Graffiti', 'id': 6, 'name': 'Graffiti'}
    1084:red:{'supercategory': 'Text', 'id': 12, 'name': 'Text'}
    1085:green:{'supercategory': 'Text', 'id': 12, 'name': 'Text'}
